In [ ]:
import numpy as np                      #For Multiple Image Detection With Summary
import tkinter as tk
from tkinter import filedialog, Scrollbar, Canvas
from PIL import Image, ImageTk, ImageOps
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('path\\best_model_95_71%.h5')

class EcoliDetectionApp:
    def __init__(self, window, window_title, image_size=(224, 224), padding=10):
        self.window = window
        self.window.title(window_title)
        self.image_size = image_size
        self.padding = padding

        # Calculate how many images can fit in a row based on the window width
        self.images_per_row = self.window.winfo_screenwidth() // (self.image_size[0] + self.padding)

        # Create a scrollable canvas for displaying multiple images and predictions
        self.canvas_frame = tk.Frame(window)
        self.canvas_frame.pack(fill=tk.BOTH, expand=1)

        self.canvas = Canvas(self.canvas_frame, bg="white")
        self.canvas.pack(side=tk.LEFT, fill=tk.BOTH, expand=1)

        self.scrollbar = Scrollbar(self.canvas_frame, orient=tk.VERTICAL, command=self.canvas.yview)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.scrollbar = Scrollbar(self.canvas_frame, orient=tk.VERTICAL, command=self.canvas.yview)
        self.scrollbar.pack(side=tk.RIGHT, fill=tk.Y)

        self.h_scrollbar = Scrollbar(window, orient=tk.HORIZONTAL, command=self.canvas.xview)
        self.h_scrollbar.pack(side=tk.BOTTOM, fill=tk.X)

        self.canvas.configure(yscrollcommand=self.scrollbar.set)
        self.canvas.bind('<Configure>', lambda e: self.canvas.configure(scrollregion=self.canvas.bbox("all")))

        self.btn_load = tk.Button(window, text="Load Images", command=self.load_images)
        self.btn_load.pack(side=tk.LEFT)

        self.label_summary = tk.Label(window, text="Summary: 0 E. coli, 0 Not E. coli")
        self.label_summary.pack(side=tk.BOTTOM, fill=tk.X)

        self.images = []  # Store loaded images for prediction
        self.window.mainloop()

    def load_images(self):
        file_paths = filedialog.askopenfilenames()  # Allows multiple file selection
        if not file_paths:
            return

        self.images = []  # Reset the list for a new set of images
        row, col = 0, 0  # Initialize row and column counters
        for file_path in file_paths:
            img = Image.open(file_path).convert('RGB')
            img_thumbnail = ImageOps.fit(img, self.image_size, Image.Resampling.LANCZOS)
            img_tk = ImageTk.PhotoImage(img_thumbnail)

            # Calculate position for the next image
            x_position = self.padding + col * (self.image_size[0] + self.padding)
            y_position = self.padding + row * (self.image_size[1] + self.padding + 20)  # Add space for prediction text

            # Display the image on the canvas
            self.canvas.create_image(x_position, y_position, anchor=tk.NW, image=img_tk)

            self.images.append((img_thumbnail, img_tk, x_position, y_position + self.image_size[1]))

            col += 1
            if col >= self.images_per_row:
                col = 0
                row += 1

        self.predict()  # Predict after all images are loaded

    def predict(self):
        e_coli_count = 0
        not_e_coli_count = 0

        for img_thumbnail, img_tk, x_position, y_text_position in self.images:
            img_array = np.array(img_thumbnail) / 255.0
            img_array = img_array.reshape((1, self.image_size[0], self.image_size[1], 3))
            prediction = model.predict(img_array)
            predicted_class = 'Not E. coli' if prediction[0][0] > 0.5 else 'E. coli'

            # Display prediction text underneath the image
            self.canvas.create_text(x_position + self.image_size[0] // 2, y_text_position, text=predicted_class, anchor=tk.N, font=("Arial", 12))

            # Count the predictions
            if predicted_class == 'E. coli':
                e_coli_count += 1
            else:
                not_e_coli_count += 1

        # Update the summary label with the counts
        self.label_summary.config(text=f"Summary: {e_coli_count} E. coli, {not_e_coli_count} Not E. coli")

EcoliDetectionApp(tk.Tk(), "E. coli Detection App")


1/1 [==============================] - 0s 87ms/step


In [26]:
import numpy as np                      #For Single Image Detection
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk, ImageOps
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('C:\\Users\\SIU856562486\\OneDrive - Southern Illinois University\\Desktop\\ResNet18\\best_model.h5')

class EcoliDetectionApp:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)

        self.canvas = tk.Canvas(window, width=224, height=224)
        self.canvas.pack()

        self.btn_load = tk.Button(window, text="Load Image", command=self.load_image)
        self.btn_load.pack(side=tk.LEFT)

        self.btn_predict = tk.Button(window, text="Predict", command=self.predict, state='disabled')
        self.btn_predict.pack(side=tk.RIGHT)

        self.label_prediction = tk.Label(window, text="Prediction: None")
        self.label_prediction.pack()

        self.img = None
        self.window.mainloop()

    def load_image(self):
        file_path = filedialog.askopenfilename()
        if not file_path:
            return

        self.img = Image.open(file_path).convert('RGB')
        self.img_thumbnail = ImageOps.fit(self.img, (224, 224), Image.Resampling.LANCZOS)  # Updated resampling method
        self.img_tk = ImageTk.PhotoImage(self.img_thumbnail)
        self.canvas.create_image(20, 20, anchor=tk.NW, image=self.img_tk)
        self.btn_predict.config(state='normal')

    def predict(self):
        img_array = np.array(self.img_thumbnail) / 255.0
        img_array = img_array.reshape((1, 224, 224, 3))  # Ensure the input shape matches the model's expected input
        prediction = model.predict(img_array)
        predicted_class = 'Not E. coli' if prediction[0][0] > 0.5 else 'E. coli'
        self.label_prediction.config(text=f"Prediction: {predicted_class}")

EcoliDetectionApp(tk.Tk(), "E. coli Detection App")

